## Extrai Tweets

In [3]:
# Instalando Bibliotecas
#!pip install tweepy datetime

## Play

In [ ]:
!python get_tweets.py

## Entendo o formato JSON

In [1]:
# Importando biblioteca
import json

In [2]:
# Carregando o arquivos com os Tweets
with open('collected_tweets2020-11-24-21-01-04.txt','r') as file:
    tweets = file.readlines()

In [3]:
# Fazendo parse do tweet para um formato json e exporta arquivo
with open('tweets.json','w') as f:
    json.dump(
        json.loads(json.loads(tweets[0])), 
        f
    )

In [ ]:
json.loads(
    json.loads(tweets[0])
)

In [6]:
parsed_tweets = [json.loads(json.loads(i)) for i in tweets]

In [7]:
# Quantidade de Tweets coletados
len(parsed_tweets)

18453

## Acessando as chaves do JSON (dicionário)

In [8]:
primeiro_tweet = parsed_tweets[1]

In [9]:
list(primeiro_tweet.items())[1]

('id', 1331387365638729731)

In [10]:
# Acessar texto do Tweet
# Chave de primero nível (Root)
primeiro_tweet['text']

"RT @GlennKesslerWP: We've updated the Trump Claims Database! He's now at 23,035 false or misleading claims as of Sept. 11. That means 525 a…"

In [11]:
# Acessar o screen_name do User
primeiro_tweet['user']["screen_name"]

'hypatiaerotica'

In [12]:
# Informação do Tweet original
primeiro_tweet['retweeted_status']["created_at"]

'Tue Nov 24 21:05:13 +0000 2020'

In [13]:
# Buscando o nome do primeiro Usuário mencionado
primeiro_tweet['retweeted_status']['entities']['user_mentions']

[]

## Transformar os Tweets em uma estrutura relacional analisável

In [14]:
import pandas as pd

In [ ]:
teste = pd.DataFrame(primeiro_tweet)
teste
# Não funciona

In [15]:
df_tratado = pd.DataFrame(primeiro_tweet).reset_index(drop=True).iloc[:1]

In [16]:
df_tratado.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'],
      dtype='object')

In [17]:
df_tratado.drop(
    columns=['quote_count','reply_count','retweet_count','favorite_count','favorited','retweeted','user','retweeted_status'], 
    inplace=True
)

In [18]:
df_tratado['user_id'] = primeiro_tweet['user']['id']
df_tratado['user_id_str'] = primeiro_tweet['user']['id_str']
df_tratado['user_screen_name'] = primeiro_tweet['user']['screen_name']
df_tratado['user_location'] = primeiro_tweet['user']['location']
df_tratado['user_description'] = primeiro_tweet['user']['description']
df_tratado['user_protected'] = primeiro_tweet['user']['protected']
df_tratado['user_verified'] = primeiro_tweet['user']['verified']
df_tratado['user_followers_count'] = primeiro_tweet['user']['followers_count']
df_tratado['user_friends_count'] = primeiro_tweet['user']['friends_count']
df_tratado['user_created_at'] = primeiro_tweet['user']['created_at']

In [19]:
df_tratado

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_id,user_id_str,user_screen_name,user_location,user_description,user_protected,user_verified,user_followers_count,user_friends_count,user_created_at
0,Wed Nov 25 00:00:58 +0000 2020,1331387365638729731,1331387365638729731,RT @GlennKesslerWP: We've updated the Trump Cl...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,...,4815996935,4815996935,hypatiaerotica,None,"Author of steamy, sensual erotic romance/eroti...",False,False,1384,814,Fri Jan 15 17:07:57 +0000 2016


In [20]:
# Entidades
primeiro_tweet['entities']

{'hashtags': [],
 'urls': [],
 'user_mentions': [{'screen_name': 'GlennKesslerWP',
   'name': 'Glenn Kessler',
   'id': 235196204,
   'id_str': '235196204',
   'indices': [3, 18]}],
 'symbols': []}

In [21]:
primeiro_tweet['entities']['user_mentions']

[{'screen_name': 'GlennKesslerWP',
  'name': 'Glenn Kessler',
  'id': 235196204,
  'id_str': '235196204',
  'indices': [3, 18]}]

In [22]:
user_mentions = []

for i in range(len(primeiro_tweet['entities']['user_mentions'])):
    dicionario_base = primeiro_tweet['entities']['user_mentions'][i].copy()
    dicionario_base.pop('indices', None)
    df = pd.DataFrame(dicionario_base, index=[0])
    df = df.rename(columns={
        'screen_name':'entities_screen_name',
        'name': 'entities_name',
        'id': 'entities_id',
        'id_str': 'entities_id_str'
    })
    user_mentions.append(df)

In [23]:
user_mentions

[  entities_screen_name  entities_name  entities_id entities_id_str
 0       GlennKesslerWP  Glenn Kessler    235196204       235196204]

In [24]:
dfs = []
for i in user_mentions:
    dfs.append(
        pd.concat([df_tratado.copy(), i], axis=1)
    )

In [25]:
pd.concat(dfs, ignore_index=True)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_description,user_protected,user_verified,user_followers_count,user_friends_count,user_created_at,entities_screen_name,entities_name,entities_id,entities_id_str
0,Wed Nov 25 00:00:58 +0000 2020,1331387365638729731,1331387365638729731,RT @GlennKesslerWP: We've updated the Trump Cl...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,...,"Author of steamy, sensual erotic romance/eroti...",False,False,1384,814,Fri Jan 15 17:07:57 +0000 2016,GlennKesslerWP,Glenn Kessler,235196204,235196204


In [26]:
def tweet_para_df(tweet):
    try:
        df_tratado = pd.DataFrame(primeiro_tweet).reset_index(drop=True).iloc[:1]
        df_tratado.drop(columns=['quote_count','reply_count','retweet_count','favorite_count','favorited','retweeted','user','retweeted_status'], inplace=True)
        df_tratado['user_id'] = tweet['user']['id']
        df_tratado['user_id_str'] = tweet['user']['id_str']
        df_tratado['user_screen_name'] = tweet['user']['screen_name']
        df_tratado['user_location'] = tweet['user']['location']
        df_tratado['user_description'] = tweet['user']['description']
        df_tratado['user_protected'] = tweet['user']['protected']
        df_tratado['user_verified'] = tweet['user']['verified']
        df_tratado['user_followers_count'] = tweet['user']['followers_count']
        df_tratado['user_friends_count'] = tweet['user']['friends_count']
        df_tratado['user_created_at'] = tweet['user']['created_at']

        user_mentions = []
        for i in range(len(tweet['entities']['user_mentions'])):
            dicionario_base = tweet['entities']['user_mentions'][i].copy()
            dicionario_base.pop('indices', None)
            df = pd.DataFrame(dicionario_base, index=[0])
            df = df.rename(columns={
                'screen_name':'entities_screen_name',
                'name': 'entities_name',
                'id': 'entities_id',
                'id_str': 'entities_id_str'
            })
            user_mentions.append(df)

        dfs = []
        for i in user_mentions:
            dfs.append(pd.concat([df_tratado.copy(), i], axis=1))
        df_final = pd.concat(dfs, ignore_index=True)
    except:
        return None
    return df_final

In [27]:
tweet_para_df(parsed_tweets[2])

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_description,user_protected,user_verified,user_followers_count,user_friends_count,user_created_at,entities_screen_name,entities_name,entities_id,entities_id_str
0,Wed Nov 25 00:00:58 +0000 2020,1331387365638729731,1331387365638729731,RT @GlennKesslerWP: We've updated the Trump Cl...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,...,None,False,False,74,352,Thu Aug 27 00:41:57 +0000 2015,robbystarbuck,Robby Starbuck,16523905,16523905
1,Wed Nov 25 00:00:58 +0000 2020,1331387365638729731,1331387365638729731,RT @GlennKesslerWP: We've updated the Trump Cl...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,None,None,None,None,...,None,False,False,74,352,Thu Aug 27 00:41:57 +0000 2015,realDonaldTrump,Donald J. Trump,25073877,25073877


In [28]:
%%time
# Vamos iterar e transformar os Tweets
parseados = [tweet_para_df(tweet) for tweet in parsed_tweets]

Wall time: 3min


In [30]:
# Eliminar posições vazias
parseados = [i for i in parseados if i is not None]

In [31]:
len(parseados)

15571

In [32]:
tratado = pd.concat(parseados, ignore_index=True)

In [ ]:
tratado

## Ingestão de dados do twitter no SQL Server

In [36]:
#!pip install pyodbc 
!pip install sqlalchemy

In [ ]:
import pyodbc 
import sqlalchemy